# Emotion Recognition using a CNN model with integrated heatmap/prioritised convolution operation

Importing the libraries I will be using

In [1]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [2]:
class DataPreprocessor:
    """
    Handles loading, preprocessing, and splitting the FER2013 dataset.
    """

    def __init__(self, dataset_path="fer2013_data"):
        self.dataset_path = dataset_path
        self.train_data = None
        self.train_labels = None
        self.val_data = None
        self.val_labels = None
        self.test_data = None
        self.test_labels = None

    def load_data_from_folder(self, folder_path):
        """
        Load images and their corresponding labels from a folder structure.
        Each subfolder name is treated as the label.
        """
        data = []
        labels = []

        for label, emotion_dir in enumerate(os.listdir(folder_path)):
            emotion_folder = os.path.join(folder_path, emotion_dir)

            if os.path.isdir(emotion_folder):
                for image_file in os.listdir(emotion_folder):
                    image_path = os.path.join(emotion_folder, image_file)

                    try:
                        # Load the image and convert it to an array
                        img = load_img(image_path, target_size=(48, 48), color_mode="grayscale")
                        img_array = img_to_array(img) / 255.0  # Normalize to [0, 1]
                        data.append(img_array)
                        labels.append(label)
                    except Exception as e:
                        print(f"Error loading image {image_path}: {e}")

        return np.array(data), np.array(labels)

    def preprocess_data(self):
        """
        Preprocess the data: load train/test folders and split train into train/validation sets.
        """
        # Load train and test data
        train_folder = os.path.join(self.dataset_path, "train")
        test_folder = os.path.join(self.dataset_path, "test")

        train_data, train_labels = self.load_data_from_folder(train_folder)
        test_data, test_labels = self.load_data_from_folder(test_folder)

        # Split train data into train/validation sets
        X_train, X_val, y_train, y_val = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

        # Convert labels to one-hot encoding
        num_classes = len(np.unique(train_labels))
        y_train = to_categorical(y_train, num_classes)
        y_val = to_categorical(y_val, num_classes)
        y_test = to_categorical(test_labels, num_classes)

        self.train_data, self.train_labels = X_train, y_train
        self.val_data, self.val_labels = X_val, y_val
        self.test_data, self.test_labels = test_data, y_test

In [3]:
class EmotionCNN:
    """
    Defines the CNN model for emotion detection and handles training and evaluation.
    """

    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.model = None

    def build_model(self):
        self.model = models.Sequential([
            layers.Conv2D(32, (3, 3), activation="relu", input_shape=self.input_shape),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(64, (3, 3), activation="relu"),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(128, (3, 3), activation="relu"),
            layers.MaxPooling2D((2, 2)),
            layers.Flatten(),
            layers.Dense(128, activation="relu"),
            layers.Dense(self.num_classes, activation="softmax"),
        ])

    def compile_model(self):
        self.model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

    def train_model(self, train_data, train_labels, val_data, val_labels, epochs):
        self.model.fit(train_data, train_labels, validation_data=(val_data, val_labels), epochs=epochs)

    def evaluate_model(self, test_data, test_labels):
        loss, accuracy = self.model.evaluate(test_data, test_labels)
        print(f"Test Accuracy: {accuracy:.2f}")
        return accuracy

    def predict_emotion(self, input_image):
        prediction = self.model.predict(np.expand_dims(input_image, axis=0))
        return np.argmax(prediction)

In [4]:
class HeatmapGenerator:
    """
    Generates and visualizes heatmaps using Class Activation Maps (CAMs).
    """

    def __init__(self, model):
        self.model = model

    def compute_gradients(self, input_image, predicted_class):
        grad_model = tf.keras.models.Model(
            [self.model.inputs], [self.model.get_layer(index=-2).output, self.model.output]
        )
        with tf.GradientTape() as tape:
            conv_outputs, predictions = grad_model(np.expand_dims(input_image, axis=0))
            loss = predictions[:, predicted_class]
        grads = tape.gradient(loss, conv_outputs)
        return conv_outputs, grads

    def generate_heatmap(self, conv_outputs, grads):
        pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
        heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_outputs[0]), axis=-1)
        heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
        return heatmap

    def visualize_heatmap(self, input_image, heatmap):
        plt.imshow(input_image.squeeze(), cmap="gray")
        plt.imshow(heatmap, cmap="jet", alpha=0.5)
        plt.colorbar()
        plt.show()

In [ ]:
class EmotionDetectionSystem:
    """
    Main class that orchestrates the entire process: preprocessing, training, and heatmap generation.
    """

    def __init__(self, dataset_path, input_shape, num_classes):
        self.dataset_path = dataset_path
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.data_preprocessor = DataPreprocessor(dataset_path)
        self.emotion_cnn = EmotionCNN(input_shape, num_classes)
        self.heatmap_generator = None

    def run(self):
        # Load and preprocess the dataset
        self.data_preprocessor.preprocess_data()

        # Build, compile, and train the CNN model
        self.emotion_cnn.build_model()
        self.emotion_cnn.compile_model()
        self.emotion_cnn.train_model(
            self.data_preprocessor.train_data,
            self.data_preprocessor.train_labels,
            self.data_preprocessor.val_data,
            self.data_preprocessor.val_labels,
            epochs=10,
        )

        # Evaluate the model on the test set
        self.emotion_cnn.evaluate_model(self.data_preprocessor.test_data, self.data_preprocessor.test_labels)

        # Generate and visualize heatmaps
        self.heatmap_generator = HeatmapGenerator(self.emotion_cnn.model)
        for i in range(3):  # Visualize for first 3 test samples
            input_image = self.data_preprocessor.test_data[i]
            predicted_class = self.emotion_cnn.predict_emotion(input_image)
            conv_outputs, grads = self.heatmap_generator.compute_gradients(input_image, predicted_class)
            heatmap = self.heatmap_generator.generate_heatmap(conv_outputs, grads)
            self.heatmap_generator.visualize_heatmap(input_image, heatmap)

: 

In [ ]:
# Run the system
dataset_path = "fer2013_data"
input_shape = (48, 48, 1)
num_classes = 7
emotion_system = EmotionDetectionSystem(dataset_path, input_shape, num_classes)
emotion_system.run()

Epoch 1/10
